<a href="https://colab.research.google.com/github/GenAIHub/agents-workshop/blob/main/03_agent_supervisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Agent Supervisor

In below example, we will create an agent group, with an agent supervisor to help delegate tasks.

![diagram](https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/img/supervisor-diagram.png?raw=1)

To simplify the code in each agent node, we will use the AgentExecutor class from LangChain.

Before we build, let's configure our environment:

In [ ]:
%%capture --no-stderr
%pip install -U langgraph
%pip install -U langchain langchain_openai langchain_experimental langsmith pandas
%pip install -U tavily-python

### Set API keys

In [ ]:
import os

# Set environment variables
os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://app-ads-sbx-openai-sw.openai.azure.com"
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-4o"

os.environ["TAVILY_API_KEY"] = ""

### Initialize the Azure LLM

In [ ]:
from langchain_openai import AzureChatOpenAI

# Fetching environment variables
api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

if not all([api_key, endpoint, api_version, deployment_name]):
    raise ValueError("One or more environment variables are missing.")

# Initialize the Azure LLM
llm = AzureChatOpenAI(
    openai_api_key=api_key,
    azure_endpoint=endpoint,
    azure_deployment=deployment_name,
    openai_api_version=api_version,
)

### Create tools

For this example, we will make an agent to do web research with a search engine, and one agent to create plots by executing python code. Define the tools they'll use below:

In [ ]:
import subprocess
import sys
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import StructuredTool

from langchain_community.tools.tavily_search import TavilySearchResults

# Import the prebuilt Tavily Tool
tavily_tool = TavilySearchResults(max_results=5)


# Let's build the code executor agent ourselves!

# This executes code locally, which can be unsafe
def execute_python_code(code):
    try:
        # Create a dictionary to capture the local variables
        local_vars = {}
        
        # Execute the code in the current notebook environment
        exec(code, globals(), local_vars)
        
        # If no string were to be returned, our supervisor would have no idea whether or not the code execution was successful.
        return "Done plotting figure."
    except Exception as e:
        return str(e)

#### **Question (optional):**

Create a tool from the function described above. <br>
You may base yourself on how we build the dall-e tool in the previous notebook.

In [ ]:
### Input Model
# To be implemented

### Tool
# To be implemented

#### **Solution:**

In [ ]:
class CodeExecutorInput(BaseModel):
    code: str = Field(
        description="The python code to be executed."
    )

python_repl_tool = StructuredTool.from_function(
    func=execute_python_code,
    # The name does not have to be the function name, but make sure that it is a relevant name as the LLM also makes use of it to decide its course of action.
    name="execute_python_code",
    # It is important to specify that the code should display the plot. Otherwise the generated code may try to save the plot instead.
    description="Execute Python code to display plots.",
    args_schema=CodeExecutorInput,
    return_direct=True
)

### Helper Utilities

Define a helper function below, which make it easier to add new agent worker nodes.

In [ ]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

def create_agent(llm: AzureChatOpenAI, tools: list, system_prompt: str):
    # The prompt should include: the system prompt, the chat history and an overview of all available tools that can be used by the agent.
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    # Prebuilt tool to create an openai agent with tools.
    # This agent generates a step-by-step plan that can be executed with the AgentExecutor.
    # The AgentExecutor itself will handle the logic to call the correct tool, no need for a seperate 'tools' node.
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor


We can also define a function that we will use to be the nodes in the graph - it takes care of converting the agent response to a human message. This is important because that is how we will add it the global state of the graph

In [ ]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

### Create Agent Supervisor

It will use function calling to choose the next worker node OR finish processing.

In [ ]:
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define members
members = ["Researcher", "Coder"]

# Define the system prompt
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)

# Define the routing options
options = ["FINISH"] + members

# Define the function that the supervisor will use.
# In this case we directly define the function in its JSON representation.
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}

# Create the supervisor prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
# Fix the options variable and the members variable defined in the system_prompt above.
).partial(options=str(options), members=", ".join(members))

# Create the supervisor chain using the Azure LLM
# Chains will call each function in the chain with the output of the function that comes before them in the chain.
# The first function will be called with the input that is given when the chain is invoked.
supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)


### Construct Graph

We're ready to start building the graph. Below, define the state and worker nodes using the function we just defined.

In [ ]:
import functools
import operator
from typing import Sequence, TypedDict
from langgraph.graph import END, StateGraph, START
from langchain_core.messages import BaseMessage, HumanMessage
from typing_extensions import Annotated

# Define the AgentState
# We add a next element to our state as the output of our supervisor will automatically overwrite this whenever it called 
# due to the function (function_def) we bound to it earlier.
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str

# Create the agents
research_agent = create_agent(llm, [tavily_tool], "You are a web researcher.")
# Fix the agent and name arguments of the agent_node functions. This way it can be invoked only using the 'state' later on.
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

code_agent = create_agent(
    llm,
    [python_repl_tool],
    "You may generate safe Python code to analyze data and generate charts using matplotlib.",
)
code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

# Build the workflow
workflow = StateGraph(AgentState)

# Add nodes
# Each agent is a seperate node
workflow.add_node("Researcher", research_node)
workflow.add_node("Coder", code_node)
workflow.add_node("supervisor", supervisor_chain)

# Add edges
# Whenever either the Coder or Researcher agent is finished, it should return to the supervisor agent.
for member in members:
    workflow.add_edge(member, "supervisor")

# Define conditional routing from supervisor
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
# Look at the 'next' in our state and route the flow to said agent.
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)

# Add the entry point
workflow.add_edge(START, "supervisor")

# Compile the graph (do this only once after all nodes and edges are added)
graph = workflow.compile()


### Invoke the team

The following example should let the system look up the current populations in LA and NY by using the Research Agent. Afterward, it should plot and display these values with the Coder Agent. Feel free to try other queries!

In [ ]:
message = "Wat is the current population in LA? Generate and display a plot that compares said population to the current population of NY."

for s in graph.stream(
    {"messages": [HumanMessage(content=message)]},
    {"recursion_limit": 100},
):
    if "__end__" not in s:
        print(s)
        print("----")
